In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation, Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
from imutils import paths
from sklearn.model_selection import train_test_split

In [8]:
dir_path = '/content/drive/MyDrive/archive/seg_train/seg_train'
categories = ['street','mountain','sea']
data = []
N = 600 #600장씩만 사용

for i in categories:

    paths = os.path.join(dir_path,i)
    print('paths',paths)
    count = 0
    for j in os.listdir(paths):
        img_path = os.path.join(paths, j)
        labels = categories.index(i)
        ext = os.path.splitext(img_path)[1].lower()
        if ext in ['.jpg', '.jpeg', '.png']:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (64, 64))
                data.append([img, labels])
                count += 1
            if count >= N:
                break  # 600장만 추가

random.shuffle(data)
print(len(data))

paths /content/drive/MyDrive/archive/seg_train/seg_train/street
paths /content/drive/MyDrive/archive/seg_train/seg_train/mountain
paths /content/drive/MyDrive/archive/seg_train/seg_train/sea
1800


정규화

In [9]:
import numpy as np
X = np.array([i[0] for i in data])
y = np.array([i[1] for i in data])
X = X.astype('float32') / 255.0

In [10]:
x = []
y = []

for features,label in data:
    x.append(features)
    y.append(label)

#Converting lists into numpy arrays
x = np.array(x)
y = np.array(y)
x = x/255.0
x = np.array(x).reshape(-1, 64, 64, 3)

#데이터 증강
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # 검증 데이터 자동 분리
)

train_gen = datagen.flow(x, y, batch_size=32, subset='training')
val_gen = datagen.flow(x, y, batch_size=32, subset='validation')

print("Shape of train images is:", x.shape)
print("Shape of labels is:", y.shape)
print(y)
print(x.shape[1:])

Shape of train images is: (1800, 64, 64, 3)
Shape of labels is: (1800,)
[1 0 0 ... 2 2 0]
(64, 64, 3)


In [11]:
model = Sequential()
model.add(Conv2D(32,(3,3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3,activation='softmax'))
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       294,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392,579 (1.50 MB)

 Trainable params: 392,579 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=70
)

Epoch 1/70


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45/45 ━━━━━━━━━━━━━━━━━━━━ 15s 279ms/step - accuracy: 0.3388 - loss: 1.1312 - val_accuracy: 0.4778 - val_loss: 1.0702
Epoch 2/70
45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 269ms/step - accuracy: 0.4563 - loss: 1.0383 - val_accuracy: 0.5278 - val_loss: 0.8757
Epoch 3/70
45/45 ━━━━━━━━━━━━━━━━━━━━ 19s 224ms/step - accuracy: 0.5735 - loss: 0.8651 - val_accuracy: 0.6056 - val_loss: 0.7557
Epoch 4/70
45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 267ms/step - accuracy: 0.5801 - loss: 0.7535 - val_accuracy: 0.5917 - val_loss: 0.7109
Epoch 5/70
45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 268ms/step - accuracy: 0.6350 - loss: 0.6953 - val_accuracy: 0.6333 - val_loss: 0.6506
Epoch 6/70
45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 275ms/step - accuracy: 0.6452 - loss: 0.6789 - val_accuracy: 0.6833 - val_loss: 0.6308
Epoch 7/70
45/45 ━━━━━━━━━━━━━━━━━━━━ 13s 285ms/step - accuracy: 0.6478 - loss: 0.6714 - val_accuracy: 0.6944 - val_loss: 0.6002
Epoch 8/70
45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 264ms/step - accuracy: 0.6898 - loss: 0.6166 - val_accuracy: 0.702

In [13]:

import numpy as np


def image(path):
    print('path',path)
    img = cv2.imread(path)

    new_arr = cv2.resize(img, (64, 64))
    new_arr = np.array(new_arr)
    new_arr = new_arr/255.0
    new_arr = new_arr.reshape(-1, 64, 64, 3)

    return new_arr



In [14]:
dir_path = '/content/drive/MyDrive/archive/seg_test/seg_test'
categories = ['street','mountain','sea']
data = []

correct_cnt=0
N = 100 #요소별 장수
for i in categories:
    paths = os.path.join(dir_path,i)
    files = os.listdir(paths)
    files = files[:N]  # 앞에서 N장만 사용
    for j in files:
        img_path = os.path.join(paths,j)
      #  print(img_path)
        labels = categories.index(i)
      #  print(img_path,labels)
        prediction = model.predict(image(img_path))
        print('labels' ,labels)
        print('classification results' ,prediction.argmax())
        if (labels==prediction.argmax()):
            correct_cnt=correct_cnt+1
print('### correnct cnt out of 100==>',correct_cnt)

path /content/drive/MyDrive/archive/seg_test/seg_test/street/20090.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20158.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20209.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20183.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20184.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20080.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20126.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
labels 0
classification results